# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

In [5]:
import perc
import default
import sys
import time
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

def perc_train(train_data, tagset, numepochs):
    feat_vec = defaultdict(int)
    default_tag = tagset[0]
    
    for epoch in range(numepochs):
        
        count_mistake = 0
        
        tic = time.time()
        
        for _, (labeled_list,feat_list) in enumerate(train_data):
            pred_output = perc.perc_test(feat_vec, labeled_list, feat_list, tagset, default_tag)
            true_output = [x.split()[2] for x in labeled_list]
            
            if pred_output != true_output:
                count_mistake += 1
                feat_index = 0
                
                for w_index in range(len(pred_output)):
                    pred_tag = pred_output[w_index]
                    true_tag = true_output[w_index]
                    (feat_index, feats) = perc.feats_for_word(feat_index, feat_list)
                    if pred_tag != true_tag:
                        for feat in feats:
                            feat_vec[feat, true_tag] += 1
                            feat_vec[feat, pred_tag] -= 1
        toc = time.time()
        print(f'Epoch {epoch+1} finished. Time cost on this epoch: {toc-tic}. Number of mistakes: {count_mistake}.')
    
    return feat_vec

tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset, 10)
perc.perc_write_to_file(feat_vec, "baseline.model")
print("wrote model to disk")

reading data ...
done.


Epoch 0 finished. Time cost on this epoch: 60.86331105232239. Number of mistakes: 6185.
Epoch 1 finished. Time cost on this epoch: 60.46869230270386. Number of mistakes: 4792.
Epoch 2 finished. Time cost on this epoch: 60.677168130874634. Number of mistakes: 3856.
Epoch 3 finished. Time cost on this epoch: 62.10835361480713. Number of mistakes: 3104.
Epoch 4 finished. Time cost on this epoch: 62.70000433921814. Number of mistakes: 2538.
Epoch 5 finished. Time cost on this epoch: 63.42802929878235. Number of mistakes: 2196.
Epoch 6 finished. Time cost on this epoch: 62.51510810852051. Number of mistakes: 1779.
Epoch 7 finished. Time cost on this epoch: 62.27873468399048. Number of mistakes: 1542.
Epoch 8 finished. Time cost on this epoch: 61.342052936553955. Number of mistakes: 1411.
Epoch 9 finished. Time cost on this epoch: 67.52357983589172. Number of mistakes: 1181.
wrote model to disk


In [6]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("baseline.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [7]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5886; correct phrases: 5331
             ADJP: precision:  58.33%; recall:  70.71%; F1:  63.93; found:    120; correct:     99
             ADVP: precision:  70.14%; recall:  76.73%; F1:  73.29; found:    221; correct:    202
            CONJP: precision:  66.67%; recall:  40.00%; F1:  50.00; found:      3; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:  90.70%; recall:  92.80%; F1:  91.73; found:   3096; correct:   3026
               PP: precision:  96.69%; recall:  97.95%; F1:  97.31; found:   1237; correct:   1221
              PRT: precision:  76.92%; recall:  45.45%; F1:  57.14; found:     13; correct:     22
             SBAR: precision:  83.17%; recall:  78.50%; F1:  80.77; found:    101; correct:    107
               VP: precision:  91.87%; recall:  91.45%; F1:  91.66; found:   1095; correct:   1100
accura